# SMAP

In [ ]:
# Import relevant libraries
import ee
import pandas as pd
ee.Authenticate()
ee.Initialize()

In [ ]:
# define region (Either place point coords, or use region defined above)
region = []

# Define the date range and load SMAP data from Earth Engine
### NB NOTE!!!: SMAP ET data only available from April 2015) ###
start_date = '2015-04-01'
end_date = '2023-09-01'

smap = ee.ImageCollection("NASA/SMAP/SPL4SMGP/007")

In [ ]:
# Filter the SMAP dataset by region and date
filtered_smap = smap.filterBounds(region).filterDate(start_date, end_date)


In [ ]:
# Define a function to calculate monthly ET sums
def calculate_monthly_et_sum(image_collection, date_str):
    """Calculates the monthly sum of ET for a given image collection and date.

    Args:
        image_collection (ee.ImageCollection): The image collection to process.
        date_str (str): The date string in 'YYYY-MM' format.

    Returns:
        dict: A dictionary containing the date and monthly ET sum.
    """
    monthly_smap_et = image_collection.select('land_evapotranspiration_flux').sum().reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=11000,
        maxPixels=1e9
    )
    return {
        'Date': date_str,
        'Monthly_ET': monthly_smap_et.get('land_evapotranspiration_flux').getInfo()
    }

# Define a function to calculate monthly sums
def calculate_monthly_sums(collection):
    """Calculates monthly sums of ET for the given collection.

    Args:
        collection (ee.ImageCollection): The image collection to process.

    Returns:
        list: A list of dictionaries containing the date and monthly ET sum for each month.
    """
    start = ee.Date(start_date)
    end = ee.Date(end_date)
    months = ee.List.sequence(0, end.difference(start, 'months').subtract(1))
    results = []

    def monthly_composite(month_offset):
        month_start = start.advance(month_offset, 'months')
        month_end = month_start.advance(1, 'months')
        date_str = month_start.format('YYYY-MM').getInfo()
        monthly_images = collection.filterDate(month_start, month_end)
        et_result = calculate_monthly_et_sum(monthly_images, date_str)
        results.append(et_result)

    for month_offset in months.getInfo():
        monthly_composite(month_offset)

    return results

# Calculate the monthly ET sums (Total ET for each month in kg/m2/s and therefore still needs to be converted below)
monthly_et_smap_results = calculate_monthly_sums(filtered_smap)

# Convert the results to a DataFrame
monthly_smap_et_df = pd.DataFrame(monthly_et_smap_results)

# Display the DataFrame
print(monthly_smap_et_df)

In [ ]:
# Apply conversion factor (kg/m2/s -> mm) and scaling factor (0.1) to get ET in mm/month:

# Multiply the 'monthly_ET' column by conversion factor
monthly_smap_et_df['Monthly_ET'] = monthly_smap_et_df['Monthly_ET'] * 86400 #(24*60*60 = 86400)

# Multiply the 'monthly_ET' column by scaling factor
monthly_smap_et_df['Monthly_ET'] = monthly_smap_et_df['Monthly_ET'] * 0.1

# Display the updated DataFrame
print(monthly_smap_et_df)

# Convert the result to a list to export it as a DataFrame
et_smap_list = monthly_smap_et_df['Monthly_ET'].tolist()
print(et_smap_list)
dates_smap_list = monthly_smap_et_df['Date'].tolist()
print(dates_smap_list)


In [ ]:
# Combine dates and ET data into a DataFrame
et_smap_df = pd.DataFrame({
    'date': dates_smap_list,
    'SMAP_ET': et_smap_list
})

# Export the DataFrame to CSV
et_smap_df.to_csv('monthly_smap_data.csv', index=False)
print("CSV file has been created.")